In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, to_date, when

Create Spark Session

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 19:30:36 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/19 19:30:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 19:30:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Convert shooting data from csv to parquets

In [4]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/landing/shooting_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/landing/shooting_data.parquet"
df.write.parquet(parquet_output_path)

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/landing/shooting_data.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

Size of initial shooting data

In [5]:
size = df.count()
print("Size of initial import:", size)

Size of initial import: 27312


Number of Features of Initial Import

In [8]:
# Count the number of columns in the DataFrame
num_columns = len(df.columns)
print("Number of columns:", num_columns)

Number of columns: 21


Removing unneccesary columns from shooting data

In [5]:
# Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/landing/shooting_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Select the columns you want to keep and rename them
selected_columns = [
    "INCIDENT_KEY as ID",
    "OCCUR_DATE as Date",
    "BORO as Borough",
    "STATISTICAL_MURDER_FLAG as Murder",
    "PERP_SEX as Perp_Sex",
    "PERP_RACE as Perp_Race",
    "VIC_SEX as Vict_Sex",
    "VIC_RACE as Vict_Race",    
    "Latitude",
    "Longitude"
]
df_selected = df.selectExpr(*selected_columns)

# Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/landing/shooting_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/landing/shooting_data_reduced.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

Restrict date between 2016 and 2019. Also note that all instances have valid latitude and longitude.

In [7]:
# Provide the path to the Parquet file
parquet_file_path = "../data/landing/shooting_data_reduced.parquet"

# Read the Parquet file into a Spark DataFrame
shooting_df = spark.read.parquet(parquet_file_path)

# Convert the 'Date' column to a date type using to_date()
shooting_df  = shooting_df .withColumn("Date", to_date(col("Date"), "MM/dd/yyyy"))

# Filter the DataFrame to get shootings between 2016 and 2019
shooting_df  = shooting_df .filter(col("Date").between("2016-01-01", "2019-12-31"))

+---------+----------+-------------+-----------------+------+--------------------+--------+--------------+--------+--------------+------------------+------------------+
|       ID|      Date|      Borough|Jurisdiction_Code|Murder|            Location|Perp_Sex|     Perp_Race|Vict_Sex|     Vict_Race|          Latitude|         Longitude|
+---------+----------+-------------+-----------------+------+--------------------+--------+--------------+--------+--------------+------------------+------------------+
|165264362|2017-05-27|       QUEENS|                0| false|                null|    null|          null|       M|         BLACK| 40.69855836100004|-73.73531116999999|
|151624171|2016-03-28|        BRONX|                0| false|MULTI DWELL - APT...|       F|BLACK HISPANIC|       M|         BLACK| 40.89018055400004|-73.85747035499998|
|174579538|2018-02-08|STATEN ISLAND|                0| false|                null|       M|BLACK HISPANIC|       M|         BLACK| 40.64075466000003|-74.07

Size of restricted shooting data (2016-2019)

In [9]:
size = shooting_df.count()
print("Size of filtered data (2016-2019):", size)

Size of filtered data (2016-2019): 4103


Convert Murder False/True to 0/1

In [11]:
# Convert "true" to 1 and "false" to 0 in the "murder" column
shooting_df_filtered = shooting_df.withColumn(
    "murder",
    when(col("murder") == True, 1)
    .when(col("murder") == False, 0)
    .otherwise(None)  # Handling any other values that might be present
)

# Show the resulting DataFrame
shooting_df_filtered.show()

+---------+----------+-------------+-----------------+------+--------------------+--------+--------------+--------+--------------+------------------+------------------+
|       ID|      Date|      Borough|Jurisdiction_Code|murder|            Location|Perp_Sex|     Perp_Race|Vict_Sex|     Vict_Race|          Latitude|         Longitude|
+---------+----------+-------------+-----------------+------+--------------------+--------+--------------+--------+--------------+------------------+------------------+
|165264362|2017-05-27|       QUEENS|                0|     0|                null|    null|          null|       M|         BLACK| 40.69855836100004|-73.73531116999999|
|151624171|2016-03-28|        BRONX|                0|     0|MULTI DWELL - APT...|       F|BLACK HISPANIC|       M|         BLACK| 40.89018055400004|-73.85747035499998|
|174579538|2018-02-08|STATEN ISLAND|                0|     0|                null|       M|BLACK HISPANIC|       M|         BLACK| 40.64075466000003|-74.07

Find date for 7 days after shooting

In [13]:
shooting_df_filtered = shooting_df_filtered.withColumn("Date_7_days_after", col("Date") + F.expr("INTERVAL 7 days"))

# Provide the path to save the filtered DataFrame to a new Parquet file
filtered_parquet_output_path = "../data/raw/shooting_data_2016-2019.parquet"

# Write the filtered DataFrame to a new Parquet file
shooting_df_filtered.write.parquet(filtered_parquet_output_path)

# Stop the SparkSession
spark.stop()

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/raw/shooting_data_2016-2019.parquet already exists. Set mode as "overwrite" to overwrite the existing path.